In [2]:
import os
import json
from dotenv import load_dotenv
import openrouteservice
from typing import TypedDict, List, Dict, Any

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq

from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from pydantic import BaseModel, Field

load_dotenv()

# Model for Agent 1
llm_fast = ChatGroq(model="llama-3.1-8b-instant", temperature=0)
print("Model 1: Llama 3 8B (Groq) Initialized.")

class KeywordList(BaseModel):
    keywords: List[str] = Field(description="A list of 5-7 specific, creative search queries.")

llm_fast_structured = llm_fast.with_structured_output(KeywordList)
print("   > Bound 'llm_fast' to a structured output (KeywordList).")

# Model for Agent 2
llm_gemini = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
print("Model 2: Gemini 2.5 Flash Initialized.")

# Model for Agent 3 
llm_hero = ChatGroq(model="llama-3.3-70b-versatile", temperature=0)
print("Model 3: Llama 3 70B (Groq) Initialized.")

try:
    ors_key = os.environ["ORS_API_KEY"]
    ors_client = openrouteservice.Client(key=ors_key)
    print("OpenRouteService Client Initialized.")
except KeyError:
    print("ERROR: ORS_API_KEY not found. Please check your .env file.")

c:\Users\comra\anaconda3\envs\agent-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model 1: Llama 3 8B (Groq) Initialized.
   > Bound 'llm_fast' to a structured output (KeywordList).
Model 2: Gemini 2.5 Flash Initialized.
Model 3: Llama 3 70B (Groq) Initialized.
OpenRouteService Client Initialized.


In [ ]:
@tool
def query_ors_places(query: str, location_name: str):
    """
    Searches for a specific place name (POI) using OpenRouteService.
    'query' is the name (e.g., 'Daunt Books').
    'location_name' is the city (e.g., 'London, UK').
    Returns name, address, and coordinates.
    """
    print(f"--- TOOL CALL: Searching for '{query}' near '{location_name}' ---")
    
    try:
        # First, find the "Focus Point" (Center of the City)
        # We use pelias_search because 'geocode' does not exist on this client.
        city_result = ors_client.pelias_search(text=location_name)
        
        if not city_result['features']:
            print(f"   > Error: City '{location_name}' not found.")
            return []
            
        # Get city center coordinates [lon, lat]
        city_coords = city_result['features'][0]['geometry']['coordinates']
        
        # Search for the place, focusing on the city center
        # This ensures we find "Daunt Books" in London, UK, not elsewhere.
        search_result = ors_client.pelias_search(
            text=query,
            focus_point=city_coords, 
            size=1
        )
        
        results = []
        for place in search_result.get('features', []): 
            props = place.get('properties', {})
            coords = place.get('geometry', {}).get('coordinates')
            
            results.append({
                "name": props.get('name', query),
                "address": props.get('label', 'Address not found'),
                "coordinates": coords
            })
            
        if not results:
            # Fallback: Try searching without the focus point if strict search fails
            print(f"   > Retrying '{query}' globally...")
            fallback_result = ors_client.pelias_search(text=f"{query}, {location_name}")
            for place in fallback_result.get('features', [])[:1]:
                props = place.get('properties', {})
                coords = place.get('geometry', {}).get('coordinates')
                results.append({
                    "name": props.get('name', query),
                    "address": props.get('label', 'Address not found'),
                    "coordinates": coords
                })

        return results

    except Exception as e:
        print(f"   > API Error: {e}")
        return []

@tool
def get_ors_directions(start_coords: List[float], end_coords: List[float], profile: str = "foot-walking"):
    """
    Gets directions between two locations.
    """
    print(f"--- TOOL CALL: get_ors_directions(start={start_coords}, end={end_coords}) ---")
    try:
        directions_result = ors_client.directions(
            coordinates=[start_coords, end_coords],
            profile=profile
        )
        if directions_result['routes']:
            summary = directions_result['routes'][0]['summary']
            return {
                "duration_minutes": round(summary.get('duration', 0) / 60, 1),
                "distance_km": round(summary.get('distance', 0) / 1000, 1)
            }
        return "No directions found."
    except Exception as e:
        return f"Error using OpenRouteService API: {e}"

# Bind Tools 
llm_gemini_tools = llm_gemini.bind_tools([query_ors_places])
llm_hero_tools = llm_hero.bind_tools([get_ors_directions])
print("Tools defined and bound.")

Tools defined and bound.


In [4]:
class TravelGraphState(TypedDict):
    destination: str
    duration_days: int
    vibe: str
    user_feedback: str
    places_to_avoid: List[str]
    keywords: List[str]
    search_results: List[dict] # Will store the full place objects
    itinerary_draft: str

print("Agent State defined.")

Agent State defined.


In [ ]:
# AGENT NODE DEFINITIONS

def vibe_interpreter_agent(state: TravelGraphState):
    """Agent 1: Uses Llama 3 70B (Hero) to generate REAL, SPECIFIC PLACE NAMES."""
    print("--- 1. EXECUTING: Vibe Interpreter (Llama 3.3 70B) ---")
    
    # We switch to the HERO model here because it knows real world places better.
    # We bind it to the structured output on the fly.
    llm_hero_structured = llm_hero.with_structured_output(KeywordList)
    
    prompt = f"""
    You are a local travel expert for {state['destination']}.
    The user wants a '{state['vibe']}' vibe and wants to avoid: {state['places_to_avoid']}.
    
    Task: Generate a list of 5-7 **REAL, SPECIFIC VENUE NAMES** that match this vibe.
    
    RULES:
    1. Return ONLY the specific name of the establishment (e.g. "Daunt Books").
    2. Do NOT include the city name (e.g. do NOT say "Daunt Books London").
    3. Do NOT return generic categories (e.g. do NOT say "Cozy Cafe").
    4. Do NOT return activities (e.g. do NOT say "Walking tour").
    5. Do NOT invent names like if you are not sure they exist.
    
    Good Examples: ['The Churchill Arms', 'Daunt Books Marylebone', 'Gordon\'s Wine Bar', 'The London Library', 'Keats House']
    Bad Examples: ['Old Pub', 'Bookstores in London', 'Literary Tours', 'Cozy Cafes']
    """
    
    response_object = llm_hero_structured.invoke(prompt)
    keywords = response_object.keywords 
    print(f"   > Generated Places: {keywords}")
    return {"keywords": keywords}

def search_agent(state: TravelGraphState):
    """Agent 2: Uses Gemini to geocode the specific place names."""
    print("--- 2. EXECUTING: Search Agent (Gemini 1.5 Flash) ---")
    
    location_fixed = state['destination']
    if "London" in location_fixed and "UK" not in location_fixed:
        location_fixed = "London, UK"
    
    search_prompt = f"""
    You are a search assistant.
    The user has provided a list of specific places in {location_fixed}: {state['keywords']}.
    
    You MUST use the `query_ors_places` tool to find the coordinates for EACH place in the list.
    
    RULES:
    1. Call the tool for EVERY keyword in the list. Do not stop if one fails.
    2. 'query': Use the specific name (e.g., "Daunt Books").
    3. 'location_name': Use "{location_fixed}".
    """
    
    response = llm_gemini_tools.invoke(search_prompt)
    
    all_places = []
    if not response.tool_calls:
        return {"search_results": []}
        
    for tool_call in response.tool_calls:
        if tool_call['name'] == 'query_ors_places':
            tool_output = query_ors_places.invoke(tool_call['args'])
            if isinstance(tool_output, list):
                all_places.extend(tool_output)

    final_places = [p for p in all_places if p['name'] not in state['places_to_avoid']]
    print(f"   > Found coordinates for {len(final_places)} places.")
    return {"search_results": final_places}

def itinerary_agent(state: TravelGraphState):
    """Agent 3: Uses Llama 3 70B to build the itinerary."""
    print("--- 3. EXECUTING: Itinerary Agent (Llama 3 70B) ---")
    
    prompt = f"""
    You are an expert travel planner for {state['destination']}.
    
    Here is a list of specific places found for the user, with their addresses:
    {json.dumps(state['search_results'], indent=2)}
    
    Task: Create a logical {state['duration_days']}-day itinerary using THESE places.
    
    Instructions:
    1. Group places that are in the same neighborhood or area (look at the addresses).
    2. Write a nice, engaging plan in Markdown.
    3. Do NOT complain about distances or locations. Assume the search results are correct.
    4. For each day, list the places to visit and explain why they fit the '{state['vibe']}' vibe.
    """
    
    # We call invoke() directly (without tools) to ensure we get TEXT back, not a tool call.
    # The agent has all the info it needs in the prompt.
    response = llm_hero.invoke(prompt)
    
    print("   > Generated Itinerary Draft.")
    return {"itinerary_draft": response.content}

In [ ]:
# Define the "Pause" and "Loop" logic

def await_feedback(state: TravelGraphState):
    """
    This node pauses the graph and waits for human feedback.
    """
    print("4. PAUSED: Awaiting User Feedback")
    # We clear the feedback so we don't loop forever
    return {"user_feedback": None}

def check_feedback(state: TravelGraphState):
    """
    This conditional edge checks if new feedback was provided.
    If YES, it loops back to the start.
    If NO, it ends the process.
    """
    print("5. CHECKING FEEDBACK")
    if state.get("user_feedback"):
        print("   > Feedback detected. Looping back to Interpreter.")
        return "vibe_interpreter" # Go back to the start
    else:
        print("   > No feedback. Plan is final.")
        return END # End the graph

# Build the Graph

workflow = StateGraph(TravelGraphState)

# Add all our agents and nodes
workflow.add_node("vibe_interpreter", vibe_interpreter_agent)
workflow.add_node("search_agent", search_agent)
workflow.add_node("itinerary_agent", itinerary_agent)
workflow.add_node("await_feedback", await_feedback)

# Define the edges (the flow)
workflow.set_entry_point("vibe_interpreter")
workflow.add_edge("vibe_interpreter", "search_agent")
workflow.add_edge("search_agent", "itinerary_agent")
workflow.add_edge("itinerary_agent", "await_feedback")

# Define the conditional loop
workflow.add_conditional_edges(
    "await_feedback",     # Start from the pause node
    check_feedback,       # Run this check function
    {
        "vibe_interpreter": "vibe_interpreter", # If it returns "vibe_interpreter", go there
        END: END                                # If it returns "END", finish
    }
)

# Compile the graph
app = workflow.compile()
print("LangGraph Compiled and Ready!")

LangGraph Compiled and Ready!


In [8]:
# This dictionary will hold the graph's state
current_state = {}

# User Input
initial_input = {
    "destination": "Visakhapatnam, India",
    "duration_days": 3,
    "vibe": "cozy, literary, night life, adventure",
    "places_to_avoid": [],
    "user_feedback": "Start"
}

# Run the Graph
for event in app.stream(initial_input):
    # event looks like: {'node_name': {'key': 'value'}}
    for node_name, node_output in event.items():
        # We merge the node output into our main state
        if isinstance(node_output, dict):
            current_state.update(node_output)
    
    # Check if we hit the pause node
    if "await_feedback" in event:
        print("\n--- ITINERARY DRAFT 1.0 ---")
        # Now 'itinerary_draft' will definitely be in current_state
        print(current_state.get('itinerary_draft', "No draft generated (Error in Itinerary Agent)"))
        break

--- 1. EXECUTING: Vibe Interpreter (Llama 3.3 70B) ---
   > Generated Places: ['The Lounge', 'Novotel Hotel', 'Dolphin Hotel', 'New Theoretical', 'The Park Hotel', 'Sea N Sand', 'Mumbai Lounge']
--- 2. EXECUTING: Search Agent (Gemini 1.5 Flash) ---
--- TOOL CALL: Searching for 'The Lounge' near 'Visakhapatnam, India' ---
   > API Error: 403 ({'error': 'Access to this API has been disallowed'})
--- TOOL CALL: Searching for 'Novotel Hotel' near 'Visakhapatnam, India' ---
   > API Error: 403 ({'error': 'Access to this API has been disallowed'})
--- TOOL CALL: Searching for 'Dolphin Hotel' near 'Visakhapatnam, India' ---
   > API Error: 403 ({'error': 'Access to this API has been disallowed'})
--- TOOL CALL: Searching for 'New Theoretical' near 'Visakhapatnam, India' ---
   > API Error: 403 ({'error': 'Access to this API has been disallowed'})
--- TOOL CALL: Searching for 'The Park Hotel' near 'Visakhapatnam, India' ---
   > API Error: 403 ({'error': 'Access to this API has been disallowed